<a href="https://colab.research.google.com/github/Dimas0824/Machine_Learning/blob/main/Jobsheet_6/Week7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Praktikum 6


1. Lakukan percobaan penggunaan ANNOY, FAISS, dan HNSWLIB pada dataset sekunder berukuran besar (Micro Spotify) pada link berikut: https://www.kaggle.com/datasets/bwandowando/spotify-songs-with-attributes-and-lyrics/data . Download data dan load CSV filenya (pilih dataset yg pertama dari dua dataset). pilih hanya fitur numerik saja, dan lakukan normalisasi menggunakan StandardScaler. Lakukan pencarian track terdekat dan

In [1]:
!pip install faiss-cpu
#!pip install faiss-gpu

In [2]:
# Karena dataset teralu besar untuk di unduh dari kaggle, maka menggunakan API kaggle merupakan solusi
from google.colab import files
files.upload()  # Upload kaggle.json

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"muhammadirsyaddimasa","key":"635e291ff03855181c5fad804eb1f37a"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d bwandowando/spotify-songs-with-attributes-and-lyrics

Dataset URL: https://www.kaggle.com/datasets/bwandowando/spotify-songs-with-attributes-and-lyrics
License(s): CC-BY-NC-SA-4.0
 98% 876M/894M [00:07<00:00, 40.3MB/s]
100% 894M/894M [00:07<00:00, 123MB/s] 


In [5]:
# Unzip dataset
!unzip /content/spotify-songs-with-attributes-and-lyrics.zip -d /content/

Archive:  /content/spotify-songs-with-attributes-and-lyrics.zip
  inflating: /content/songs_with_attributes_and_lyrics.csv  
  inflating: /content/songs_with_lyrics_and_timestamps.csv  


In [4]:
import pandas as pd

df = pd.read_csv('/content/songs_with_attributes_and_lyrics.csv')

# Lihat data awal
df.head()

# Lihat data akhir
df.tail()

# Shape
df.shape

# Jumlah data
len(df)

# Info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 955320 entries, 0 to 955319
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                955320 non-null  object 
 1   name              955309 non-null  object 
 2   album_name        385557 non-null  object 
 3   artists           955318 non-null  object 
 4   danceability      955320 non-null  float64
 5   energy            955320 non-null  float64
 6   key               955320 non-null  object 
 7   loudness          955320 non-null  float64
 8   mode              955320 non-null  object 
 9   speechiness       955320 non-null  float64
 10  acousticness      955320 non-null  float64
 11  instrumentalness  955320 non-null  float64
 12  liveness          955320 non-null  float64
 13  valence           955320 non-null  float64
 14  tempo             955320 non-null  float64
 15  duration_ms       955320 non-null  float64
 16  lyrics            95

In [5]:
!pip install annoy hnswlib

In [1]:
import pandas as pd
import numpy as np
import time
import faiss
from annoy import AnnoyIndex
import hnswlib
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

# -------------------------------
# Load dataset
# -------------------------------
df = pd.read_csv('/content/songs_with_attributes_and_lyrics.csv')
features = ['danceability', 'energy', 'loudness', 'speechiness',
            'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
X = df[features].values

In [3]:
# Standarisasi fitur
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

k = 10  # jumlah nearest neighbors

print("Step selesai")

Step selesai


In [4]:
X_sample = X_scaled[:100000]
print("Scale dengan subset data untuk mempercepat proses")

Scale dengan subset data untuk mempercepat proses


In [12]:
# -------------------------------
# Exact Nearest Neighbor (brute-force)
# -------------------------------
start = time.time()
nn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='euclidean')
nn.fit(X_sample)
dist_exact, idx_exact = nn.kneighbors(X_sample)
time_exact = time.time() - start
print(f"Exact NN done in {time_exact:.3f} s")

Exact NN done in 62.830 s


In [13]:
# -------------------------------
# Annoy
# -------------------------------
start = time.time()
f = X_sample.shape[1]
index_annoy = AnnoyIndex(f, 'euclidean')
for i, v in enumerate(X_sample):
    index_annoy.add_item(i, v)
index_annoy.build(10)
idx_annoy = [index_annoy.get_nns_by_vector(v, k) for v in X_sample]
time_annoy = time.time() - start
print(f"Annoy done in {time_annoy:.3f} s")

Annoy done in 5.153 s


In [5]:
# -------------------------------
# HNSW
# -------------------------------
start = time.time()
p_hnsw = hnswlib.Index(space='l2', dim=X_sample.shape[1])
p_hnsw.init_index(max_elements=X_sample.shape[0], ef_construction=200, M=16)
p_hnsw.add_items(X_sample)
p_hnsw.set_ef(200)
idx_hnsw, dist_hnsw = p_hnsw.knn_query(X_sample, k=k)
time_hnsw = time.time() - start
print(f"HNSW done in {time_hnsw:.3f} s")

HNSW done in 31.612 s


In [9]:
# -------------------------------
# FAISS IVF
# -------------------------------
start = time.time()

d = X_sample.shape[1]

quantizer = faiss.IndexFlatL2(d)  # base index (exact search untuk clustering)
index_faiss = faiss.IndexIVFFlat(quantizer, d, 100, faiss.METRIC_L2)  # gunakan positional arg
index_faiss.train(X_sample)  # train clustering centroids (nlist=100)
index_faiss.add(X_sample)    # add all vectors ke index
index_faiss.nprobe = 10      # jumlah cluster yang di-scan saat query

dist_faiss, idx_faiss = index_faiss.search(X_sample, k)  # query subset agar cepat
time_faiss = time.time() - start

print(f"FAISS IVF done in {time_faiss:.3f} s")
print("Neighbors:", idx_faiss[0][:5])

FAISS IVF done in 11.145 s
Neighbors: [    0 61511 85956  3836 35205]


In [14]:
# -------------------------------
# Contoh tampilkan top-5 neighbors dari item pertama
# -------------------------------
print("\nTop-5 neighbors for first song:")
print(f"Exact NN: {idx_exact[0][:5]}")
print(f"Annoy:    {idx_annoy[0][:5]}")
print(f"HNSW:     {idx_hnsw[0][:5]}")
print(f"FAISS:    {idx_faiss[0][:5]}")


Top-5 neighbors for first song:
Exact NN: [    0 61511 85956  3836 35205]
Annoy:    [0, 61511, 85956, 3836, 35205]
HNSW:     [    0 61511 85956  3836 35205]
FAISS:    [    0 61511 85956  3836 35205]
